In [8]:
import pandas as pd
import numpy as np
import os
import datetime

In [9]:
import git
import sys
repo = git.Repo("./", search_parent_directories=True)
homedir = repo.working_dir

In [10]:
#Standard Mobility DL data
Mobility = pd.read_csv(f"{homedir}/data/us/mobility/DL-us-mobility-daterow.csv", encoding='latin1')
Mobility = Mobility.drop(columns = ['country_code','admin_level','admin1','admin2'])
Mobility = Mobility[Mobility['fips'] > 1000]
Mobility.to_csv('DL_Mobility.csv')

In [11]:
#Key, that has fips code and 'loc' [County Name, State_Abbreviation]
Key = pd.read_csv(f"{homedir}/data/us/processing_data/fips_key.csv", encoding='latin1')
Key = Key.drop(columns=['MSA/PMSA NECMA']) #Dropping unecessary column
Key = Key.append({'FIPS':12027,'ST' :'FL','COUNTY' :'Desoto County'}, ignore_index = True)
Key = Key.append({'FIPS':8014,'ST' :'CO','COUNTY' :'Broomfield County'}, ignore_index = True)
Key['loc'] = Key.COUNTY.astype(str).str.cat(Key.ST.astype(str), sep=', ')


In [12]:
#Dictionary mapping state names to state abbreviations
Abbrev = pd.read_json(f"{homedir}/data/us/processing_data/po_code_state_map.json")
Abbrev = np.array(Abbrev)
#Dictionary from state names to state initials
State_Dict = dict((rows[2],rows[1]) for rows in Abbrev)

In [13]:
#Global Google Mobility Data
Google = pd.read_csv(f"{homedir}/data/google_mobility/Global_Mobility_Report.csv", encoding='latin1')
Google = Google[Google['country_region_code'] == 'US']
Google = Google[~pd.isna(Google['sub_region_2'])]
Google = Google.replace({'sub_region_1': State_Dict})
#Fixing some specific county names
Google.loc[Google['sub_region_2'] == 'LaPorte County', 'sub_region_2'] = 'Laporte County'
Google.loc[Google['sub_region_2'] == 'LaGrange County', 'sub_region_2'] = 'Lagrange County'
Google.loc[Google['sub_region_2'] == 'LaSalle County', 'sub_region_2'] = 'Lasalle County'
Google.loc[Google['sub_region_2'] == 'DeSoto County', 'sub_region_2'] = 'Desoto County'
Google.loc[Google['sub_region_2'] == 'DuPage County', 'sub_region_2'] = 'Dupage County'
Google.loc[Google['sub_region_2'] == 'DoÃ±a Ana County', 'sub_region_2'] = 'Dona Ana County'

#Fixing some issues with bad county names
Google['loc'] = Google.sub_region_2.astype(str).str.cat(Google.sub_region_1.astype(str), sep=', ')
Google['loc_county'] = Google.sub_region_2.astype(str).str.cat(Google.sub_region_1.astype(str), sep=' County, ')
Google['loc_city'] = Google.sub_region_2.astype(str).str.cat(Google.sub_region_1.astype(str), sep=' city, ')
Google['loc_census'] = Google.sub_region_2.astype(str).str.cat(Google.sub_region_1.astype(str), sep=' Borough, ')
Google['loc_borough'] = Google.sub_region_2.astype(str).str.cat(Google.sub_region_1.astype(str), sep=' Census Area, ')

Google_1 = pd.merge(Google, Key, left_on='loc', right_on='loc', sort=True)
Google_2 = pd.merge(Google, Key, left_on='loc_county', right_on='loc', sort=True)
Google_3 = pd.merge(Google, Key, left_on='loc_city', right_on='loc', sort=True)
Google_4 = pd.merge(Google, Key, left_on='loc_census', right_on='loc', sort=True)
Google_5 = pd.merge(Google, Key, left_on='loc_borough', right_on='loc', sort=True)
Frame = [Google_1, Google_2, Google_3, Google_4, Google_5]

Google = pd.concat(Frame, sort = True)
Google = Google.drop_duplicates(subset = ['FIPS','date'])

Google = Google.drop(columns = ['COUNTY','ST','country_region','country_region_code','loc','loc_x','loc_y', \
                               'loc_borough','loc_census','loc_city','loc_county','sub_region_1',\
                                'sub_region_2']).sort_values(['FIPS','date']).rename(columns={'date':'Date'}).reset_index(drop=True)
Google.to_csv('google_new.csv')

C:\Users\jethi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
Google

,FIPS,Date,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,residential_percent_change_from_baseline,retail_and_recreation_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline
0,1001,2020-02-15,7.0,NaN,NaN,5.0,NaN,-4.0
1,1001,2020-02-16,1.0,-23.0,NaN,0.0,NaN,-4.0
2,1001,2020-02-17,0.0,NaN,5.0,8.0,NaN,-27.0
3,1001,2020-02-18,0.0,NaN,0.0,-2.0,NaN,2.0
4,1001,2020-02-19,0.0,NaN,0.0,-2.0,NaN,2.0
5,1001,2020-02-20,-3.0,NaN,2.0,-8.0,NaN,1.0
6,1001,2020-02-21,-2.0,NaN,0.0,-3.0,NaN,3.0
7,1001,2020-02-22,1.0,NaN,NaN,1.0,NaN,8.0
8,1001,2020-02-23,3.0,NaN,NaN,0.0,NaN,5.0
9,1001,2020-02-24,-5.0,NaN,1.0,-2.0,NaN,2.0
